In [2]:
# import nisnap
import pyxnat
import xnat
from pyxnat import Interface
import pandas as pd
import nibabel as nib
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# #The URL of your XNAT
# xnat_host = os.environ['XNAT_HOST']

# #A temporary token which can be used as a username
# user = os.environ['XNAT_USER']

# #A temporary token which can be used as a password
# pwd = os.environ['XNAT_PASS']

In [ ]:
# my_pyxnat = Interface(server=xnat_host,
#                     user=user, password=pwd)

In [3]:
my_pyxnat = Interface(server="https://xnat.crmbm.univ-amu.fr",
                    user='hdary')

In [4]:
# if os.environ['XNAT_XSI_TYPE'] != 'xnat:projectData':
#     raise Exception('Must be started from an XNAT project.')

# project = os.environ['XNAT_ITEM_ID']

In [5]:
project = 'TERRA_BRAIN_EPINOV'

In [6]:
def get_resources(my_pyxnat, project):

    resources = []
    experiments = my_pyxnat.array.experiments(project_id=project, columns=['ID', 'project', 'label', 'subject_ID']).data
    # For each experiments fetch all the resources associated with it
    for exp in tqdm(experiments):
        j = my_pyxnat.get('{}/{}'.format(exp['URI'], 'resources')).json()
        exp_res = j['ResultSet']['Result']

        if exp_res:
            resources.extend([[exp['project'], exp['ID'], exp['label'], exp['subject_ID'],
                               r['xnat_abstractresource_id'],
                               r['label']] for r in exp_res])
        else:
            resources.append([exp['project'], exp['ID'],  exp['label'], exp['subject_ID'],
                              'No Data', 'No Data'])
    return resources

In [7]:
def get_subjects(my_pyxnat, project):
    dico = {i.id() :i.label() for i in my_pyxnat.select.project(project).subjects()}
    return dico

In [8]:
dico = get_subjects(my_pyxnat, project)

Resources

In [9]:
resources = get_resources(my_pyxnat, project)

  0%|          | 0/34 [00:00<?, ?it/s]

100%|██████████| 34/34 [00:00<00:00, 50.85it/s]


In [10]:
dico_resources = {}
for r in resources:
    if r[5] == 'No Data':
        continue
    if r[5] in dico_resources.keys():
        dico_resources[r[5]].append(dico[r[3]])
    else:
        dico_resources[r[5]] = []
        dico_resources[r[5]].append(dico[r[3]])

In [11]:
# Créer un ensemble contenant tous les sujets
sujets = set()
for valeurs in dico_resources.values():
    sujets.update(valeurs)

# Créer une copie du dictionnaire pour conserver les noms des sujets dans les valeurs
dictionnaire_modifie = {colonne: [sujet in valeurs for sujet in sujets] for colonne, valeurs in dico_resources.items()}

# Convertir le dictionnaire modifié en DataFrame
index = list(sujets)
dataframe = pd.DataFrame(dictionnaire_modifie)

dataframe.insert(0, "Sujets", list(sujets))

# Remplacer les index par les noms des sujets
# dataframe.index = list(sujets)


In [22]:
cell_colors = [['white' if j == 0 else 'green' if val else 'red' for val in dataframe[col]] for j, col in enumerate(dataframe.columns)]

fig = go.Figure(data=[go.Table(
    header=dict(values=list(dataframe.columns)),
    cells=dict(values=[dataframe[col] for col in dataframe.columns],
               fill_color=cell_colors)
)])

# Afficher le tableau Plotly
fig.show()

Atlas

In [237]:
dico_atlas_path = {'AAL3': 'AAL3/AAL3v1_1mm_to_subj.nii.gz',
            'Neudorfer': 'Neudorfer/MDA_990HCP_atlas_labels_0.5mm.nii.gz_to_subj.nii.gz',
            'AAL3_Neudorfer': 'AAL3_Neudorfer/neudorfer_aal3_to_subj.nii.gz',
            'Hypothalamus': 'Hypothalamus/Hypothalamus2Adele_dilated2_resliced_to_subj.nii.gz',
            'atlas_7TAmi_DGN_SBA_v9': '7TAMI/7TAmi_DGN_SBA_v9_to_subj.nii.gz',
            'atlas_7TAmi_adapted_eMorel_v9': '7TAMI/7TAmi_adapted_eMorel_v9_to_subj.nii.gz',
            'atlas_7TAmi_Amygdala': '7TAMI/7TAmi_Amygdala_to_subj.nii.gz',
            'atlas_7TAmi_CIT168_AMYG': '7TAMI/7TAmi_CIT168_AMYG_to_subj.nii.gz'}

dico_atlas_freesurfer_path = {
            'VEP_atlas': 'subject/mri/origin/aparc+aseg.vep.mgz',
            'Destrieux': 'subject/mri/origin/aparc.a2009s+aseg.mgz',
            'Desikan_Killiany': 'subject/mri/origin/aparc_DKTatlas+aseg.mgz'}

dico_atlas_list = dico_atlas_path.copy()
dico_atlas_list = {cle: [] for cle in dico_atlas_list}
dico_atlas_list['session'] = []
dico_atlas_list['subjects'] = []
for k in dico_atlas_freesurfer_path.keys():
    dico_atlas_list[k] = []

dico_atlas_freesurfer_path = {
            'VEP_atlas': 'subject/mri/origin/aparc+aseg.vep.mgz',
            'Destrieux': 'subject/mri/origin/aparc.a2009s+aseg.mgz',
            'Desikan_Killiany': 'subject/mri/origin/aparc_DKTatlas+aseg.mgz'}

In [238]:
dico_atlas = {}
for sub in my_pyxnat.select.projects(project).subjects():
    for exp in sub.experiments():
        dico_atlas[sub.label() + '*' + exp.label()] = {}
        for res in exp.resources():
            if res.label() == 'FREESURFER':
                dico_atlas[sub.label() + '*' + exp.label()]['aparc'] = res.aparc()
                dico_atlas[sub.label() + '*' + exp.label()]['aseg'] = res.aseg()

                for k in dico_atlas_freesurfer_path.keys():
                    if res.files(k + '.csv').get() != []:
                        # file_object = res.files(k + '.csv')[0].attributes()
                        file_path = res.files(k + '.csv')[0].get()
                        df = pd.read_csv(file_path)
                        dico_atlas[sub.label() + '*' + exp.label()]['extraction_' + k] = df

            elif res.label() == 'Atlas':
                for k,v in dico_atlas_path.items():
                    if res.files(v).get() != []:
                        dico_atlas[sub.label() + '*' + exp.label()][k] = True

            elif res.label() == 'Extraction':
                for k in dico_atlas_path.keys():
                    if res.files(k + '.csv').get() != []:
                        # file_object = res.files(k + '.csv')[0].attributes()
                        file_path = res.files(k + '.csv')[0].get()
                        df = pd.read_csv(file_path)
                        dico_atlas[sub.label() + '*' + exp.label()]['extraction_' + k] = df

In [239]:
dico_atlas['01-59-P-V*202204131119'].keys()

dict_keys(['aparc', 'aseg', 'AAL3', 'AAL3_Neudorfer', 'atlas_7TAmi_DGN_SBA_v9', 'atlas_7TAmi_adapted_eMorel_v9', 'atlas_7TAmi_Amygdala', 'atlas_7TAmi_CIT168_AMYG', 'extraction_AAL3_Neudorfer', 'extraction_atlas_7TAmi_Amygdala', 'extraction_atlas_7TAmi_CIT168_AMYG'])

In [240]:
for sub in dico_atlas.keys():
    dico_atlas_list['subjects'].append(sub.split('*')[0])
    dico_atlas_list['session'].append(sub.split('*')[1])
    for atlas in dico_atlas_list.keys():
        if atlas == 'session' or atlas == 'subjects':
            continue
        if atlas in dico_atlas[sub].keys():
            dico_atlas_list[atlas].append(True)
        else:
            dico_atlas_list[atlas].append(False)

In [242]:
my_dataframe = pd.DataFrame(dico_atlas_list)
new_list = [x for x in list(my_dataframe.columns) if x != 'subjects' and x != 'session']
nouvel_ordre_colonnes = ['subjects', 'session'] + new_list
my_dataframe = my_dataframe[nouvel_ordre_colonnes]

In [243]:
cell_colors = [['white' if j == 0 or j == 1 else 'green' if val else 'red' for val in my_dataframe[col]] for j, col in enumerate(my_dataframe.columns)]

fig = go.Figure(data=[go.Table(
    header=dict(values=list(my_dataframe.columns)),
    cells=dict(values=[my_dataframe[col] for col in my_dataframe.columns],
               fill_color=cell_colors)
               )])

fig.show()

In [566]:
import pandas as pd
import ipywidgets as widgets
from functools import partial

# Define the list of atlas
list_of_atlas = ['AAL3', 'Neudorfer', 'AAL3_Neudorfer', 'Hypothalamus', 'atlas_7TAmi_DGN_SBA_v9', 'atlas_7TAmi_adapted_eMorel_v9', 'atlas_7TAmi_Amygdala', 'atlas_7TAmi_CIT168_AMYG', 'VEP_atlas', 'Destrieux', 'Desikan_Killiany']
list_of_subjects = sorted(list(dico_atlas.keys()))

# Create dropdown for subjects
subjects_dropdown = widgets.Dropdown(options=list_of_subjects, description='Subjects:')

# Create buttons for atlas selection
atlas_buttons = [widgets.Button(description=atlas) for atlas in list_of_atlas]

# Create output widget for table
output = widgets.Output()


# Function to handle atlas button click
def on_button_clicked(b):
    with output:
        output.clear_output()
        selected_subject = subjects_dropdown.value
        selected_atlas = b.description
        try:
            print(f"Atlas selected: {selected_atlas}")
            atlas_data = dico_atlas[selected_subject]['extraction_' + selected_atlas]
            # Display atlas data
            print(f"Atlas: {selected_atlas}\n")
            df_atlas = pd.DataFrame(atlas_data)
            # Afficher le DataFrame dans un widget Table
            display(df_atlas)
        except KeyError:
            print(f"No atlas data found for subject {selected_subject} and atlas {selected_atlas}")

# Attach button click handler to each button
for button in atlas_buttons:
    button.on_click(on_button_clicked)

# Function to handle subject dropdown change
def on_subject_dropdown_change(change):
    with output:
        output.clear_output()
        selected_subject = change.new
        print(f"Subject selected: {selected_subject}")

# Attach subject dropdown change handler
subjects_dropdown.observe(on_subject_dropdown_change, names='value')

# Display widgets
display(widgets.VBox([subjects_dropdown, widgets.HBox(atlas_buttons), output]))



Volume Freesurfer

In [ ]:
dico_aseg_list = {}
# dico_aseg_list['session'] = []
# dico_aseg_list['subjects'] = []
dico_aseg_list['values'] = []
dico_aseg_list['label'] = []

for sub in dico_atlas.keys():
    # dico_aseg_list['subjects'].append(sub.split('*')[0])
    # dico_aseg_list['session'].append(sub.split('*')[1])
    if 'aseg' in dico_atlas[sub].keys():
        # print(dico_atlas[sub]['aseg'])
        df = dico_atlas[sub]['aseg']
        # Filtrer les lignes avec measurement égal à "Volume_mm3"
        df_volume = df[df['measurement'] == 'Volume_mm3']
        # Supprimer la colonne "measurement"
        df_volume = df_volume.drop(columns=['measurement'])
        # Transposer le DataFrame
        df_transpose = df_volume.set_index('region').T
        # Réinitialiser les index pour obtenir une mise en forme correcte
        df_transpose.reset_index(drop=True, inplace=True)
        dico_aseg_list['label'].append(['subjects', 'session'] + list(df_transpose.columns))
        dico_aseg_list['values'].append([sub.split('*')[0], sub.split('*')[1]] + list(df_transpose.iloc[0]))
        print()

    else:
        # dico_aseg_list['label'].append(['subjects', 'session'] + 59*[0])
        dico_aseg_list['values'].append([sub.split('*')[0], sub.split('*')[1]] + + 59*[0])

In [472]:
my_dataframe2 = pd.DataFrame(dico_aseg_list['values'])
my_dataframe2.columns = dico_aseg_list['label'][0]

In [483]:
my_dataframe2

,subjects,session,BrainSegVol,BrainSegVolNotVent,VentricleChoroidVol,lhCortexVol,rhCortexVol,lhCerebralWhiteMatterVol,rhCerebralWhiteMatterVol,CerebralWhiteMatterVol,...,Right-WM-hypointensities,non-WM-hypointensities,Left-non-WM-hypointensities,Right-non-WM-hypointensities,Optic-Chiasm,CC_Posterior,CC_Mid_Posterior,CC_Central,CC_Mid_Anterior,CC_Anterior
0,01_55_L_M,202201140838,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,01_45_B-N,202107020849,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,01_45_B-N,BRAIN_EPINOV,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,01_85_G-S,202306090849,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,01-59-P-V,202204131119,1.022580e+06,9.801073e+05,36647.696150,204580.125985,213219.579237,196262.102864,206063.094817,402325.197681,...,0.0,0.0,0.0,0.0,144.5,311.7,184.6,165.8,172.5,136.2
5,01-68-P-N,202211080928,9.246668e+05,9.034393e+05,17805.592883,192301.355968,191040.035488,186254.064186,186594.744360,372848.808545,...,0.0,0.0,0.0,0.0,144.0,314.2,100.5,183.1,171.0,373.5
6,01-75-R-S,202303011104,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
7,01-63-B-K,202205020920,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
8,01_77_B_B,202308220902,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
9,01_86_P-L,202306291315,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [495]:
my_dataframe4 = my_dataframe2.copy()
my_dataframe4['subjects'] = my_dataframe4['subjects'].str.cat(my_dataframe4['session'], sep='*')
my_dataframe4.set_index('subjects', inplace=True)
my_dataframe4 = my_dataframe4.iloc[:, 2:]
my_dataframe4 = my_dataframe4[(my_dataframe4 != 0).any(axis=1)]
my_dataframe4 = my_dataframe4.astype(float)

In [496]:
my_dataframe4

,BrainSegVolNotVent,VentricleChoroidVol,lhCortexVol,rhCortexVol,lhCerebralWhiteMatterVol,rhCerebralWhiteMatterVol,CerebralWhiteMatterVol,SubCortGrayVol,TotalGrayVol,SupraTentorialVol,...,Right-WM-hypointensities,non-WM-hypointensities,Left-non-WM-hypointensities,Right-non-WM-hypointensities,Optic-Chiasm,CC_Posterior,CC_Mid_Posterior,CC_Central,CC_Mid_Anterior,CC_Anterior
subjects,,,,,,,,,,,,,,,,,,,,,
01-59-P-V*202204131119,9.801073e+05,36647.696150,204580.125985,213219.579237,196262.102864,206063.094817,402325.197681,44579.309386,557499.642814,907305.992304,...,0.0,0.0,0.0,0.0,144.5,311.7,184.6,165.8,172.5,136.2
01-68-P-N*202211080928,9.034393e+05,17805.592883,192301.355968,191040.035488,186254.064186,186594.744360,372848.808545,47839.710148,509031.309091,825551.687911,...,0.0,0.0,0.0,0.0,144.0,314.2,100.5,183.1,171.0,373.5
01-66-L-Q*202109090853,1.042571e+06,25576.548073,204876.401487,209801.863320,225695.507993,224281.291607,449976.799601,52740.738104,566747.807952,947874.383144,...,0.0,5.9,0.0,0.0,200.9,639.6,296.1,309.1,308.0,609.3


In [513]:

df_long = my_dataframe4.reset_index().melt(id_vars='subjects', var_name='Region', value_name='Volume')

fig = px.box(df_long, x='Region', y='Volume', hover_name='subjects', boxmode="overlay", points='all',
             title='Volume in mm3')

fig.show()

In [512]:
df_long = my_dataframe4.reset_index().melt(id_vars='subjects', var_name='Region', value_name='Volume')

# Tracer les données avec Plotly Express
fig = px.box(df_long, x='subjects', y='Volume', hover_name='Region', boxmode="overlay", points='all',
                 title='Volumes des régions par sujet', labels={'subjects': 'Sujet', 'Volume': 'Volume (mm3)'})
fig.show()

In [537]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets

# Suppose you already have your DataFrame df
df = my_dataframe4

# Define unique values for the dropdown fields
unique_subjects = sorted(df.index.unique())

# Define dropdown fields for each column of your DataFrame
subjects_dropdown = widgets.Dropdown(options=unique_subjects, description='Subjects:')

# Create output widget for the table
table_output = widgets.Output()

# Display initial table
with table_output:
    selected_subject = unique_subjects[0]
    filtered_df = df.loc[[selected_subject]]
    regions = filtered_df.columns
    volumes = filtered_df.values.flatten()
    data = {'Regions': regions, 'Volume': volumes}
    table = go.FigureWidget(data=[go.Table(
        header=dict(values=list(data.keys())),
        cells=dict(values=[data['Regions'], data['Volume']])
    )])
    table.update_layout(title='Volume by Region for Subject: ' + selected_subject)
    display(table)

# Define function to update the table based on dropdown value
def update_table(change):
    selected_subject = subjects_dropdown.value
    filtered_df = df.loc[[selected_subject]]
    regions = filtered_df.columns
    volumes = filtered_df.values.flatten()
    data = {'Regions': regions, 'Volume': volumes}
    with table_output:
        table.data[0].cells.values = [data['Regions'], data['Volume']]
        table.update_layout(title='Volume by Region for Subject: ' + selected_subject)

# Observe changes in dropdown value
subjects_dropdown.observe(update_table, names="value")

# Display dropdown and table output
display(widgets.VBox([subjects_dropdown, table_output]))


In [536]:
# import pandas as pd
# import plotly.graph_objects as go
# import ipywidgets as widgets
# from functools import reduce

# df = dataframe
# # Supposons que vous ayez déjà votre DataFrame df

# # Définir des valeurs uniques pour les champs de saisie
# sujets_uniques = sorted(df['Sujets'].unique())

# # Définir des champs de saisie pour chaque colonne de votre DataFrame
# sujets = widgets.Dropdown(options=sujets_uniques, description='Sujets:')

# # Créer un tableau Plotly initial
# initial_temp_df = df.copy()
# initial_fig = go.Figure(data=[go.Table(
#     header=dict(values=initial_temp_df.columns),
#     cells=dict(values=[initial_temp_df[col] for col in initial_temp_df.columns])
# )])

# # Créer une sortie pour le tableau Plotly
# table_plot = widgets.Output()
# with table_plot:
#     initial_fig.show(renderer="notebook")

# # Définir une fonction pour mettre à jour le tableau Plotly
# def update_plot(change):
#     # Filtrer le DataFrame en fonction des valeurs sélectionnées dans les champs de saisie
#     temp_df = df.copy()
#     filter_list = [temp_df['Sujets'] == sujets.value]
#     temp_df = temp_df[reduce(lambda x, y: x & y, filter_list)]
    
#     # Mettre à jour le tableau Plotly avec les données filtrées
#     with table_plot:
#         table_plot.clear_output(wait=True)
#         fig = go.Figure(data=[go.Table(
#             header=dict(values=temp_df.columns),
#             cells=dict(values=[temp_df[col] for col in temp_df.columns])
#         )])
#         fig.show(renderer="notebook")

# # Observer les changements dans les champs de saisie
# sujets.observe(update_plot, names="value")

# # Ajouter un bouton Reset pour réinitialiser l'affichage à tous les sujets
# reset_button = widgets.Button(description="Reset")

# def reset_plot(_):
#     with table_plot:
#         table_plot.clear_output(wait=True)
#         initial_fig.show(renderer="notebook")

# reset_button.on_click(reset_plot)

# # Afficher les champs de saisie, le bouton Reset et le tableau Plotly
# widgets.VBox([sujets, reset_button, table_plot])


KeyError: 'Sujets'

KeyError: 'Sujets'